In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q pandas numpy scikit-learn matplotlib tensorflow

In [8]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Veriyi oku (senin Drive'ındaki dosya yolu)
df = pd.read_csv('/content/drive/MyDrive/enestrade/data/merged_training_data.csv')

# İlk 5 satırı kontrol et (sorun varsa görelim)
df.head()


,open_time,open,high,low,close,volume,vwap,volume_delta,price_change_pct,Sentiment_Score,...,vwap_slope,zscore_close,candle_body,upper_wick,lower_wick,trend_direction,target,forward_ret_max,symbol,interval
0,2024-06-22 00:00:00,0.4016,0.4064,0.3956,0.4012,1783864.4,0.401067,-2026725.6,-0.149328,0.276,...,-0.004867,-0.805930,0.0004,0.0048,0.0056,0,1,0.020688,1INCHUSDT,12h
1,2024-06-22 12:00:00,0.4013,0.4119,0.4012,0.4095,1229211.3,0.407533,-554653.1,2.068794,0.276,...,0.006467,-0.279516,0.0082,0.0024,0.0001,0,0,-0.004884,1INCHUSDT,12h
2,2024-06-23 00:00:00,0.4097,0.4160,0.4013,0.4020,2236116.6,0.406433,1006905.3,-1.831502,0.276,...,-0.001100,-0.674564,0.0077,0.0063,0.0007,0,1,0.013682,1INCHUSDT,12h
3,2024-06-23 12:00:00,0.4019,0.4038,0.3873,0.3881,1789340.1,0.393067,-446776.5,-3.457711,0.276,...,-0.013367,-1.336268,0.0138,0.0019,0.0008,0,1,0.049987,1INCHUSDT,12h
4,2024-06-24 00:00:00,0.3885,0.3920,0.3705,0.3865,3749911.5,0.383000,1960571.4,-0.412265,0.276,...,-0.010067,-1.280226,0.0020,0.0035,0.0160,0,1,0.054334,1INCHUSDT,12h


In [9]:
# Hedef (target) sütununu ayır
y = df['target']

# Girdi verileri: sadece sayısal olanları al
X = df.drop(columns=['target'], errors='ignore')
X = X.select_dtypes(include=[np.number])  # sadece sayısal sütunlar

# Normalize et
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# LSTM için 3D shape: (örnek sayısı, zaman adımı, özellik sayısı)
X_scaled = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))


In [10]:
# Eğitim ve test veri kümesini ayır
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Boyutları kontrol et
print("X_train shape:", X_train.shape)
print("X_val shape:", X_val.shape)


X_train shape: (692872, 1, 42)
X_val shape: (173218, 1, 42)


In [11]:
# BiLSTM modeli
model = Sequential([
    Bidirectional(LSTM(64, return_sequences=True), input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.3),
    Bidirectional(LSTM(32)),
    Dropout(0.3),
    Dense(1, activation="sigmoid")
])

# Modeli derle
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Callbacks tanımla
model_path = "/content/drive/MyDrive/enestrade/models/bilstm_model.keras"

early_stop = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True, verbose=1)
checkpoint = ModelCheckpoint(model_path, save_best_only=True, verbose=1)

# Eğitimi başlat
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=64,
    callbacks=[early_stop, checkpoint],
    verbose=1
)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
10827/10827 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7037 - loss: 0.6034
Epoch 1: val_loss improved from inf to 0.59840, saving model to /content/drive/MyDrive/enestrade/models/bilstm_model.keras
10827/10827 ━━━━━━━━━━━━━━━━━━━━ 102s 9ms/step - accuracy: 0.7037 - loss: 0.6034 - val_accuracy: 0.7029 - val_loss: 0.5984
Epoch 2/50
10823/10827 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7041 - loss: 0.5972
Epoch 2: val_loss improved from 0.59840 to 0.59397, saving model to /content/drive/MyDrive/enestrade/models/bilstm_model.keras
10827/10827 ━━━━━━━━━━━━━━━━━━━━ 90s 8ms/step - accuracy: 0.7041 - loss: 0.5972 - val_accuracy: 0.7030 - val_loss: 0.5940
Epoch 3/50
10822/10827 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7037 - loss: 0.5939
Epoch 3: val_loss improved from 0.59397 to 0.59095, saving model to /content/drive/MyDrive/enestrade/models/bilstm_model.keras
10827/10827 ━━━━━━━━━━━━━━━━━━━━ 143s 8ms/step - accuracy: 0.7037 - loss: 0.5939 - val_accuracy: 0.7029 - va